In [52]:
import pandas as pd
import os
import tqdm
filename='../FDDC/dingzeng/dingzeng.train'

In [53]:
def getDict(name,start=-1,end=-1,sentence=-1):
    return {'name':name,'start':start,'end':end,'sentence':sentence}
class hetong():
    def __init__(self,name,addObj,addType,addNum,addPrice,lookup,buyType):
        self.name=name
        self.addObj=[]
        self.addObj.append(getDict(addObj))
        self.addType=[]
        self.addType.append(getDict(addType))
        self.addNum=[]
        self.addNum.append(getDict(addNum))
        self.addPrice=[]
        self.addPrice.append(getDict(addPrice))
        self.lookup=[]
        self.lookup.append(getDict(lookup))
        self.buyType=[]
        self.buyType.append(getDict(buyType))
    

In [54]:
length,length2,ht=[],[],[]
with open(filename,'r') as fr:
    for line in fr.readlines():
        a=line.split('\t')
        length.append(len(a))
        if len(a)<7:
            a.extend(['']*(7-len(a)))
        length2.append(len(a))
        ht.append(hetong(*a))

In [55]:
dirname='../FDDC/dingzeng/data'
if not os.path.exists(dirname):
    os.makedirs(dirname)
# for index,i in tqdm.tqdm(enumerate(ht[:-2])):
    
for file in tqdm.tqdm(sorted(list(os.walk('../FDDC/dingzeng/textWithPara'))[0][2],key=lambda x:int(x.split('.')[0]),reverse = True)[:1]):
    with open('../FDDC/dingzeng/textWithPara/'+file,'r') as fr:
        name = file.split('.')[0]
        ht_obj = []
        ht_tmp = ht[0]
        while ht_tmp.__dict__['name'] ==name:
            ht_obj.append(ht_tmp)
            ht=ht[1:]
            ht_tmp = ht[0]
        sss=""
        for i in ht_obj:
            i=i.__dict__
            if i['addObj']=='' or i['addType']=='':
                continue
            name=i['name']
            i.pop('name')
            
            text=fr.readline()
            sentence=text.split('。')
            addObj_name=i['addObj'][0]['name']
            addType_name=i['addType'][0]['name']
            addNum_name=i['addNum'][0]['name']
            addPrice_name=i['addPrice'][0]['name']
            lookup_name=i['lookup'][0]['name']
            buyType_name=i['buyType'][0]['name']

            index=0
            for num,sen in enumerate(sentence):
                addObj_start=sen.find(addObj_name)
                addType_start=sen.find(addType_name)
                if addObj_start!=-1 and addType_start!=-1:
                    i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))
                index+=len(sen)+1
            index=0
            for num,sen in enumerate(sentence):

                addObj_start=sen.find(addObj_name)
                addType_start=sen.find(addType_name)
                addNum_start=sen.find(addNum_name)
                addPrice_start=sen.find(addPrice_name)
                lookup_start=sen.find(lookup_name)
                buyType_start=sen.find(buyType_name)


                if addNum_name!='' and addNum_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['addNum'].append(getDict(addPrice_name,index+addPrice_start,index+addPrice_start+len(addPrice_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))   
                if addPrice_name!='' and addPrice_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['addPrice'].append(getDict(addPrice_name,index+addPrice_start,index+addPrice_start+len(addPrice_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num)) 
                if lookup_name!='' and lookup_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['lookup'].append(getDict(lookup_name,index+lookup_start,index+lookup_start+len(lookup_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))   
                if buyType_name!='' and buyType_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['buyType'].append(getDict(buyType_name,index+buyType_start,index+buyType_start+len(buyType_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))   
                index+=len(sen)+1

        li=['O' for i in text]
        for i in ht_obj:
            for k,v in i.items():
                for item in v:
                    if item['name']!='' and item['start']!=-1:
                        li[item['start']]='B-'+k
                        for i in range(item['start']+1,item['end']):
                            li[i]='I-'+k
        for j,con in enumerate(li):
            if con!='O':
                sub_start=j
                break

        for j,con in enumerate(li):
            if li[len(li)-j-1]!='O':
                sub_end=len(li)-j-1
                break
        li=li[sub_start-10:sub_end+10]
        text=text[sub_start-10:sub_end+10]



        for j,con in enumerate(li):
            sss+=text[j]+' '+con+'\n'
    with open(os.path.join('../FDDC/dingzeng/test',name+'.txt'),'w') as fw:
        fw.write(sss)

                

  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'hetong' object has no attribute 'items'

In [53]:
import os
import numpy as np
import random
dirname='../FDDC/dingzeng/textWithMoney'
files=list(os.walk(dirname))[0][2]
random.shuffle(files)
train="example.train"
test="example.test"
dev="example.dev"

with open(dev,'w') as fw:
    for file in files[int(len(files)*0.9):]:
        with open(os.path.join(dirname,file),'r') as fr:
            fw.writelines(fr.readlines())
        fw.write('\n')

In [30]:
sorted(list(os.walk('../FDDC/dingzeng/textWithPara'))[0][2],key=lambda x:int(x.split('.')[0]),reverse = True)

['20503293.txt',
 '20371858.txt',
 '20337881.txt',
 '20326871.txt',
 '20314486.txt',
 '20286416.txt',
 '20279155.txt',
 '20278299.txt',
 '20275260.txt',
 '20270805.txt',
 '20270527.txt',
 '20260228.txt',
 '20239739.txt',
 '20214486.txt',
 '20212396.txt',
 '20199509.txt',
 '20199507.txt',
 '19743270.txt',
 '19695420.txt',
 '19351877.txt',
 '19351874.txt',
 '19351866.txt',
 '19351860.txt',
 '19342865.txt',
 '19334507.txt',
 '19330582.txt',
 '19304784.txt',
 '19304774.txt',
 '19304289.txt',
 '19304236.txt',
 '19299347.txt',
 '19288690.txt',
 '19283783.txt',
 '19275238.txt',
 '19274647.txt',
 '19242533.txt',
 '19242471.txt',
 '19238446.txt',
 '19230597.txt',
 '19229921.txt',
 '19217629.txt',
 '19213144.txt',
 '19205580.txt',
 '19202024.txt',
 '19172706.txt',
 '19163640.txt',
 '19163639.txt',
 '18887833.txt',
 '18887599.txt',
 '18887429.txt',
 '18887420.txt',
 '18872014.txt',
 '18869299.txt',
 '18869294.txt',
 '18866051.txt',
 '18861329.txt',
 '18856777.txt',
 '18856621.txt',
 '18855239.txt

In [35]:
ht[0].__dict__,ht[1].__dict__

({'addNum': [{'end': -1, 'name': '9206511', 'sentence': -1, 'start': -1}],
  'addObj': [{'end': -1, 'name': '建信基金管理有限责任公司', 'sentence': -1, 'start': -1}],
  'addPrice': [{'end': -1,
    'name': '181000006.3',
    'sentence': -1,
    'start': -1}],
  'addType': [{'end': -1, 'name': '竞价', 'sentence': -1, 'start': -1}],
  'buyType': [{'end': -1, 'name': '', 'sentence': -1, 'start': -1}],
  'lookup': [{'end': -1, 'name': '12\n', 'sentence': -1, 'start': -1}],
  'name': '20503293'},
 {'addNum': [{'end': -1, 'name': '15208545', 'sentence': -1, 'start': -1}],
  'addObj': [{'end': -1, 'name': '金鹰基金管理有限公司', 'sentence': -1, 'start': -1}],
  'addPrice': [{'end': -1,
    'name': '298999994.7',
    'sentence': -1,
    'start': -1}],
  'addType': [{'end': -1, 'name': '竞价', 'sentence': -1, 'start': -1}],
  'buyType': [{'end': -1, 'name': '', 'sentence': -1, 'start': -1}],
  'lookup': [{'end': -1, 'name': '12\n', 'sentence': -1, 'start': -1}],
  'name': '20503293'})

In [39]:
ht.pop().__dict__

{'addNum': [{'end': -1, 'name': '7000000', 'sentence': -1, 'start': -1}],
 'addObj': [{'end': -1, 'name': '融通基金管理有限公司', 'sentence': -1, 'start': -1}],
 'addPrice': [{'end': -1, 'name': '58450000', 'sentence': -1, 'start': -1}],
 'addType': [{'end': -1, 'name': '竞价', 'sentence': -1, 'start': -1}],
 'buyType': [{'end': -1, 'name': '', 'sentence': -1, 'start': -1}],
 'lookup': [{'end': -1, 'name': '12\n', 'sentence': -1, 'start': -1}],
 'name': '7880'}

In [37]:
ht[0].__dict__,ht[1].__dict__

({'addNum': [{'end': -1, 'name': '9206511', 'sentence': -1, 'start': -1}],
  'addObj': [{'end': -1, 'name': '建信基金管理有限责任公司', 'sentence': -1, 'start': -1}],
  'addPrice': [{'end': -1,
    'name': '181000006.3',
    'sentence': -1,
    'start': -1}],
  'addType': [{'end': -1, 'name': '竞价', 'sentence': -1, 'start': -1}],
  'buyType': [{'end': -1, 'name': '', 'sentence': -1, 'start': -1}],
  'lookup': [{'end': -1, 'name': '12\n', 'sentence': -1, 'start': -1}],
  'name': '20503293'},
 {'addNum': [{'end': -1, 'name': '15208545', 'sentence': -1, 'start': -1}],
  'addObj': [{'end': -1, 'name': '金鹰基金管理有限公司', 'sentence': -1, 'start': -1}],
  'addPrice': [{'end': -1,
    'name': '298999994.7',
    'sentence': -1,
    'start': -1}],
  'addType': [{'end': -1, 'name': '竞价', 'sentence': -1, 'start': -1}],
  'buyType': [{'end': -1, 'name': '', 'sentence': -1, 'start': -1}],
  'lookup': [{'end': -1, 'name': '12\n', 'sentence': -1, 'start': -1}],
  'name': '20503293'})